In [1]:
def method(image_path):
    # disable warnings
    import warnings
    warnings.filterwarnings("ignore")

    # import modules
    import cv2
    import numpy as np
    import tensorflow as tf
    from tensorflow.contrib.layers import flatten

    def grayAndEqualizeHist(img):
        """
            :param img: input RGB image
            :return: histogram equalized grayscale image
        """
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY )
        equHist  = cv2.equalizeHist(gray)
        return equHist
    
    def ConvNet(x):    
        # Hyperparameters
        mu = 0
        sigma = 0.1

        # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
        W1 = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma), name="W1")
        x = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='VALID')
        b1 = tf.Variable(tf.zeros(6), name="b1")
        x = tf.nn.bias_add(x, b1)

        # Activation.
        x = tf.nn.relu(x)

        # Pooling. Input = 28x28x6. Output = 14x14x6.
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        layer1 = x

        # Layer 2: Convolutional. Output = 10x10x16.
        W2 = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma), name="W2")
        x = tf.nn.conv2d(x, W2, strides=[1, 1, 1, 1], padding='VALID')
        b2 = tf.Variable(tf.zeros(16), name="b2")
        x = tf.nn.bias_add(x, b2)

        # Activation.
        x = tf.nn.relu(x)

        # Pooling. Input = 10x10x16. Output = 5x5x16.
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

        # Layer 3: Convolutional. Output = 1x1x400.
        W3 = tf.Variable(tf.truncated_normal(shape=(5, 5, 16, 400), mean = mu, stddev = sigma), name="W3")
        x = tf.nn.conv2d(x, W3, strides=[1, 1, 1, 1], padding='VALID')
        b3 = tf.Variable(tf.zeros(400), name="b3")
        x = tf.nn.bias_add(x, b3)

        # Activation.
        x = tf.nn.relu(x)

        # Flatten. Input = 14x14x6. Output = 1176.
        layer1flat = flatten(layer1)

        # Flatten x. Input = 1x1x400. Output = 400.
        xflat = flatten(x)

        # Concat layer1flat and x. Input = 1176 + 400. Output = 1576.
        x = tf.concat([xflat, layer1flat], 1)

        # Dropout.
        x = tf.nn.dropout(x, keep_prob)

        # Layer 4: Fully Connected. Input = 1576. Output = 43.
        W4 = tf.Variable(tf.truncated_normal(shape=(1576, 43), mean = mu, stddev = sigma), name="W4")
        b4 = tf.Variable(tf.zeros(43), name="b4")    
        logits = tf.add(tf.matmul(x, W4), b4)

        return logits
    
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, (None, 32, 32, 1))

    keep_prob = tf.placeholder(tf.float32)

    logits = ConvNet(x)

    saver = tf.train.Saver()

    def predictImageLabel(image_path):
        model = 'ConvNet'
        image = cv2.imread(image_path)
        image_enchanced = grayAndEqualizeHist(image)
        array_image = np.zeros((1,32,32,1))
        array_image[0][:][:][:] = np.array(image_enchanced)[...,np.newaxis]
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.import_meta_graph('./' + model + '.meta')
            saver.restore(sess, "./" + model)
            label = sess.run(tf.argmax(logits, 1), feed_dict={x: array_image, keep_prob: 1.0})[0]
        return label

    return predictImageLabel(image_path)

In [3]:
print("Semnul de circulatie este:", method("real_images/2.png"))

Semnul de circulatie este: 1
